Question 2
Part 1
Read all the json files in the folder called Data.
 There are three categories of json files in this folder. They are identified by the key
called “term” in each of the json file.
 Create a folder structure to read all these json files and store them into these separate
folders. You are expected to create a hierarchy of folder structure.
 Example:
 You can place all restaurants json files in a particular country (say Australia) in the
same folder. How you group the json files and create a folder structure is your choice.
Your task is to identify criteria by which you can group all these json files and store
them.
 (You could use these keys to create hierarchy and store json files: Country, city,
categories)
 Output Format:
 Create a folder (Name: Data Processed)
 In this folder you should have a hierarchy of folder structures


Part 2
Read all the json files in the folder called Data.
 Read only the json files which contain the key called “restaurants”
 Each (or most of the json files) contain a key called “open” which contains the details
of the operation (timings) of the restaurants. For each json file, read the timings of the
restaurants.
 Data of the operation timings of the restaurants is present for each day of the week. I
want you to extract each of this data and write it in an excel sheet.
 Example:
 For a particular restaurant named “The Coffee Grounds”, the excel sheet should look
like this: (see ppt for table)

In [1]:
import glob

In [8]:
glob.glob('./Data/*')

['./Data\\1-abercrombie-lane-sydney.json',
 './Data\\1-chicken-rice-and-seafood-houston.json',
 './Data\\1-chinese-kitchen-san-jose-2.json',
 './Data\\10-custom-pizzeria-braddon.json',
 './Data\\10-downing-street-london.json',
 './Data\\10-greek-street-london.json',
 './Data\\100-club-london.json',
 './Data\\100-de-agave-denver.json',
 './Data\\100-hoxton-london.json',
 './Data\\109-adelaide.json',
 './Data\\11-inch-pizza-melbourne.json',
 './Data\\111-by-nico-glasgow-2.json',
 './Data\\127-bar-and-bistro-brisbane.json',
 './Data\\13-north-grille-phoenix.json',
 './Data\\13th-note-glasgow.json',
 './Data\\140-perth-perth.json',
 './Data\\1428-haight-patio-cafe-and-crepery-san-francisco.json',
 './Data\\143-photo-booths-san-jose.json',
 './Data\\163-vietnamese-sandwiches-and-bubble-tea-boston.json',
 './Data\\1760-san-francisco.json',
 './Data\\1806-melbourne.json',
 './Data\\18b-the-las-vegas-art-distirct-las-vegas.json',
 './Data\\1907-restaurant-perth.json',
 './Data\\1912-dining-bar